In [1]:
# CELL 1: IMPORTS & SETUP 
# ============================================================================
print("="*70)
print("CELL 1: Importing Libraries ")
print("="*70)

import os
import sys
import math
import time
import json
from collections import deque

# Built-in XML parser (no svgpathtools needed!)
import xml.etree.ElementTree as ET
import re

# NumPy and Matplotlib (pre-installed on DexterOS)
import numpy as np
import matplotlib.pyplot as plt

# SciPy for distance transform (pre-installed on DexterOS)
from scipy.ndimage import distance_transform_edt, label, find_objects

# Robot hardware (pre-installed on DexterOS)
import easygopigo3 as easy
try:
    from di_sensors.inertial_measurement_unit import InertialMeasurementUnit
except ImportError:
    print("⚠ IMU not available, will use encoders only")
    InertialMeasurementUnit = None

# Setup paths
BASE_DIR = os.path.expanduser("~")
DESIGNS_DIR = os.path.join(BASE_DIR, "parking_designs")
OUTPUTS_DIR = os.path.join(BASE_DIR, "parking_outputs")

# Create directories
os.makedirs(DESIGNS_DIR, exist_ok=True)
os.makedirs(OUTPUTS_DIR, exist_ok=True)

print(f"\n✓ Working directory: {BASE_DIR}")
print(f"✓ Designs folder: {DESIGNS_DIR}")
print(f"✓ Outputs folder: {OUTPUTS_DIR}")
print("✓ All libraries imported successfully\n")


# ============================================================================
# CELL 2: CONFIGURATION
# ============================================================================
print("="*70)
print("CELL 2: System Configuration")
print("="*70)

# === GRID & SCALING ===
GRID_WIDTH = 100
GRID_HEIGHT = 100
POINT_SPACING = 1  # Extract every point (no spacing)
GRID_CELL_SIZE_CM = 2  # Each grid cell = 10cm (ADJUSTABLE)

# === ROBOT STARTING POSITION ===
START_X = 0  # Top-left corner
START_Y = 99  # Y decreases downward
START_HEADING = 0  # Compass: 0=North, 90=East, 180=South, 270=West

# === COORDINATE SYSTEM ===
ORIGIN = "TOP_LEFT"  # (0,99) is top-left
Y_DIRECTION = "DOWNWARD"  # Y increases downward

# === PAINTING LOGIC ===
PAINT_DISTANCE_THRESHOLD = 3  # Paint if waypoints ≤3 cells apart
SKIP_DISTANCE_THRESHOLD = 3   # Skip (lift servo) if >3 cells apart

# === SERVO ANGLES ===
SERVO_PAINT_DOWN = 45  # Painting position
SERVO_PAINT_UP = 90    # Lifted position

# === ROBOT PHYSICAL PARAMETERS ===
WHEEL_DIAMETER_MM = 66.5
WHEEL_BASE_MM = 117
ENCODER_TICKS_PER_ROTATION = 360

# === NAVIGATION ===
ROBOT_SPEED_DPS = 150  # Degrees per second (~8.7 cm/sec)
POSITION_TOLERANCE_CM = 5  # ±5cm accuracy target
HEADING_TOLERANCE_DEG = 5  # ±5° heading accuracy

# === LOCALIZATION ===
USE_IMU_CONTINUOUS = False  # Use encoders only, IMU for calibration
POSITION_UPDATE_HZ = 10  # Update position 10x per second

# === OBSTACLE DETECTION ===
OBSTACLE_STOP_DISTANCE_MM = 200  # Stop if object within 20cm
OBSTACLE_CHECK_INTERVAL = 0.1  # Check every 0.1 seconds
OBSTACLE_RECOVERY_MODE = "RESUME"  # Resume from stopped position

print("\n📋 Configuration Summary:")
print(f"  Grid: {GRID_WIDTH}x{GRID_HEIGHT} cells")
print(f"  Cell size: {GRID_CELL_SIZE_CM} cm")
print(f"  Start position: ({START_X}, {START_Y}) facing North")
print(f"  Robot speed: {ROBOT_SPEED_DPS} DPS (~8.7 cm/sec)")
print(f"  Position accuracy: ±{POSITION_TOLERANCE_CM} cm")
print(f"  Obstacle threshold: {OBSTACLE_STOP_DISTANCE_MM} mm\n")


# ============================================================================
# CELL 3: HARDWARE INITIALIZATION
# ============================================================================
print("="*70)
print("CELL 3: Initializing Robot Hardware")
print("="*70)

# Initialize GoPiGo3
try:
    gpg = easy.EasyGoPiGo3()
    print("✓ GoPiGo3 initialized")
except Exception as e:
    print(f"✗ Error initializing GoPiGo3: {e}")
    gpg = None

# Initialize Servos
try:
    servo1 = gpg.init_servo("SERVO1")
    servo2 = gpg.init_servo("SERVO2")
    print("✓ Servos initialized (SERVO1, SERVO2)")
except Exception as e:
    print(f"✗ Error initializing servos: {e}")
    servo1 = None
    servo2 = None

# Initialize IMU Sensor
try:
    if InertialMeasurementUnit:
        imu = InertialMeasurementUnit(bus="GPG3_AD1")
        print("✓ IMU sensor initialized on AD1")
    else:
        imu = None
        print("⚠ IMU not available")
except Exception as e:
    print(f"✗ Error initializing IMU: {e}")
    imu = None

# Initialize Distance Sensor
try:
    distance_sensor = gpg.init_distance_sensor("I2C")
    print("✓ Distance sensor initialized on I2C")
except Exception as e:
    print(f"✗ Error initializing distance sensor: {e}")
    distance_sensor = None

print("\n✓ Hardware initialization complete\n")

CELL 1: Importing Libraries 

✓ Working directory: /home/jupyter
✓ Designs folder: /home/jupyter/parking_designs
✓ Outputs folder: /home/jupyter/parking_outputs
✓ All libraries imported successfully

CELL 2: System Configuration

📋 Configuration Summary:
  Grid: 100x100 cells
  Cell size: 2 cm
  Start position: (0, 99) facing North
  Robot speed: 150 DPS (~8.7 cm/sec)
  Position accuracy: ±5 cm
  Obstacle threshold: 200 mm

CELL 3: Initializing Robot Hardware
✓ GoPiGo3 initialized
✓ Servos initialized (SERVO1, SERVO2)
✓ IMU sensor initialized on AD1
✓ Distance sensor initialized on I2C

✓ Hardware initialization complete



In [12]:
# CELL 4: SVG FILE UPLOAD
# ============================================================================
print("="*70)
print("CELL 4: SVG File Upload")
print("="*70)

print(f"\n📁 Upload your SVG file to: {DESIGNS_DIR}")
print("\nAvailable SVG files:")

svg_files = [f for f in os.listdir(DESIGNS_DIR) if f.endswith('.svg')]
if svg_files:
    for i, f in enumerate(svg_files, 1):
        print(f"  {i}. {f}")
else:
    print("  (No SVG files found)")

print("\nTo use a file, set: svg_filename = 'parking-svgrepo-com.svg'")
print("Or copy to:", DESIGNS_DIR)
print()

# Example: svg_filename = "parking_design.svg"
# svg_path = os.path.join(DESIGNS_DIR, svg_filename)


CELL 4: SVG File Upload

📁 Upload your SVG file to: /home/jupyter/parking_designs

Available SVG files:
  1. road-highway-svgrepo-com.svg
  2. parking-svgrepo-com.svg
  3. svgviewer-output.svg

To use a file, set: svg_filename = 'parking-svgrepo-com.svg'
Or copy to: /home/jupyter/parking_designs



In [13]:
# CELL 5: SVG → POINTS EXTRACTION 
# ============================================================================
print("="*70)
print("CELL 5: Extracting Points from SVG (Built-in XML Parser)")
print("="*70)

def parse_svg_path_commands(path_string):
    """
    Parse SVG path data into coordinate pairs
    Supports: M, L, H, V, C, S, Q, T commands (absolute and relative)
    """
    points = []
    
    # Remove extra whitespace and split by commands
    path_string = re.sub(r'\s+', ' ', path_string.strip())
    
    # Extract all commands and their parameters
    # Match letters followed by numbers
    commands = re.findall(r'([MLHVCSQTZmlhvcsqtz])([^MLHVCSQTZmlhvcsqtz]*)', path_string)
    
    current_x, current_y = 0, 0
    start_x, start_y = 0, 0
    
    for cmd, params_str in commands:
        # Extract numbers (including negative and decimals)
        params = [float(x) for x in re.findall(r'[-+]?\d*\.?\d+', params_str)]
        
        if cmd == 'M':  # Move to (absolute)
            for i in range(0, len(params), 2):
                current_x, current_y = params[i], params[i+1]
                start_x, start_y = current_x, current_y
                points.append([current_x, current_y])
        
        elif cmd == 'm':  # Move to (relative)
            for i in range(0, len(params), 2):
                current_x += params[i]
                current_y += params[i+1]
                start_x, start_y = current_x, current_y
                points.append([current_x, current_y])
        
        elif cmd == 'L':  # Line to (absolute)
            for i in range(0, len(params), 2):
                current_x, current_y = params[i], params[i+1]
                points.append([current_x, current_y])
        
        elif cmd == 'l':  # Line to (relative)
            for i in range(0, len(params), 2):
                current_x += params[i]
                current_y += params[i+1]
                points.append([current_x, current_y])
        
        elif cmd == 'H':  # Horizontal line (absolute)
            for x in params:
                current_x = x
                points.append([current_x, current_y])
        
        elif cmd == 'h':  # Horizontal line (relative)
            for dx in params:
                current_x += dx
                points.append([current_x, current_y])
        
        elif cmd == 'V':  # Vertical line (absolute)
            for y in params:
                current_y = y
                points.append([current_x, current_y])
        
        elif cmd == 'v':  # Vertical line (relative)
            for dy in params:
                current_y += dy
                points.append([current_x, current_y])
        
        elif cmd in ['C', 'c']:  # Cubic Bezier curve
            # Simplified: just use endpoints
            is_relative = (cmd == 'c')
            for i in range(0, len(params), 6):
                if i + 5 < len(params):
                    x, y = params[i+4], params[i+5]
                    if is_relative:
                        current_x += x
                        current_y += y
                    else:
                        current_x, current_y = x, y
                    points.append([current_x, current_y])
        
        elif cmd in ['Z', 'z']:  # Close path
            current_x, current_y = start_x, start_y
            points.append([current_x, current_y])
    
    return points


def extract_points_from_svg(svg_file, sample_distance=5):
    """
    """
    all_points = []
    
    try:
        tree = ET.parse(svg_file)
        root = tree.getroot()
        
        print("  Parsing SVG with built-in XML parser...")
        
        # Remove namespace if present
        def remove_namespace(tag):
            return tag.split('}')[-1] if '}' in tag else tag
        
        # Parse all SVG elements
        for elem in root.iter():
            tag = remove_namespace(elem.tag)
            
            if tag == 'path':
                # Parse path data
                d = elem.get('d', '')
                if d:
                    path_points = parse_svg_path_commands(d)
                    
                    # Sample along path
                    for i in range(len(path_points) - 1):
                        x1, y1 = path_points[i]
                        x2, y2 = path_points[i+1]
                        
                        dist = math.hypot(x2 - x1, y2 - y1)
                        num_points = max(int(dist / sample_distance), 2)
                        
                        for j in range(num_points):
                            t = j / (num_points - 1)
                            x = x1 + t * (x2 - x1)
                            y = y1 + t * (y2 - y1)
                            all_points.append([x, y])
            
            elif tag == 'line':
                x1 = float(elem.get('x1', 0))
                y1 = float(elem.get('y1', 0))
                x2 = float(elem.get('x2', 0))
                y2 = float(elem.get('y2', 0))
                
                dist = math.hypot(x2 - x1, y2 - y1)
                num_points = max(int(dist / sample_distance), 2)
                
                for i in range(num_points):
                    t = i / (num_points - 1)
                    x = x1 + t * (x2 - x1)
                    y = y1 + t * (y2 - y1)
                    all_points.append([x, y])
            
            elif tag == 'rect':
                x = float(elem.get('x', 0))
                y = float(elem.get('y', 0))
                w = float(elem.get('width', 0))
                h = float(elem.get('height', 0))
                
                # Rectangle perimeter
                corners = [[x, y], [x+w, y], [x+w, y+h], [x, y+h], [x, y]]
                for i in range(len(corners) - 1):
                    x1, y1 = corners[i]
                    x2, y2 = corners[i+1]
                    dist = math.hypot(x2 - x1, y2 - y1)
                    num_points = max(int(dist / sample_distance), 2)
                    
                    for j in range(num_points):
                        t = j / (num_points - 1)
                        x_pt = x1 + t * (x2 - x1)
                        y_pt = y1 + t * (y2 - y1)
                        all_points.append([x_pt, y_pt])
            
            elif tag in ['polyline', 'polygon']:
                points_str = elem.get('points', '')
                if points_str:
                    coords = points_str.replace(',', ' ').split()
                    pts = []
                    for i in range(0, len(coords)-1, 2):
                        pts.append([float(coords[i]), float(coords[i+1])])
                    
                    # Add closing point for polygons
                    if tag == 'polygon' and len(pts) > 0:
                        pts.append(pts[0])
                    
                    # Sample between points
                    for i in range(len(pts) - 1):
                        x1, y1 = pts[i]
                        x2, y2 = pts[i+1]
                        dist = math.hypot(x2 - x1, y2 - y1)
                        num_points = max(int(dist / sample_distance), 2)
                        
                        for j in range(num_points):
                            t = j / (num_points - 1)
                            x = x1 + t * (x2 - x1)
                            y = y1 + t * (y2 - y1)
                            all_points.append([x, y])
            
            elif tag == 'circle':
                cx = float(elem.get('cx', 0))
                cy = float(elem.get('cy', 0))
                r = float(elem.get('r', 0))
                
                circumference = 2 * math.pi * r
                num_points = max(int(circumference / sample_distance), 8)
                
                for i in range(num_points):
                    angle = 2 * math.pi * i / num_points
                    x = cx + r * math.cos(angle)
                    y = cy + r * math.sin(angle)
                    all_points.append([x, y])
            
            elif tag == 'ellipse':
                cx = float(elem.get('cx', 0))
                cy = float(elem.get('cy', 0))
                rx = float(elem.get('rx', 0))
                ry = float(elem.get('ry', 0))
                
                # Approximate ellipse perimeter
                perimeter = math.pi * (3 * (rx + ry) - math.sqrt((3 * rx + ry) * (rx + 3 * ry)))
                num_points = max(int(perimeter / sample_distance), 8)
                
                for i in range(num_points):
                    angle = 2 * math.pi * i / num_points
                    x = cx + rx * math.cos(angle)
                    y = cy + ry * math.sin(angle)
                    all_points.append([x, y])
        
        print(f"  Found {len(all_points)} points")
        
    except Exception as e:
        print(f"  ✗ Error parsing SVG: {e}")
        return np.array([])
    
    return np.array(all_points) if all_points else np.array([])


# Example usage (uncomment when you have an SVG file):
# points = extract_points_from_svg(svg_path, sample_distance=5)
# print(f"\n✓ Extracted {len(points)} points from SVG\n")


# ============================================================================
# CELL 6: POINTS → GRID CONVERSION
# ============================================================================
print("="*70)
print("CELL 6: Converting Points to Grid")
print("="*70)

def points_to_grid(points, target_width=100, target_height=100, spacing=1):
    """Convert points to occupancy grid"""
    if len(points) == 0:
        raise ValueError("No points to convert!")
    
    min_x, min_y = points.min(axis=0)
    max_x, max_y = points.max(axis=0)
    
    width = target_width
    height = target_height
    
    scale_x = (max_x - min_x) / width if (max_x - min_x) > 0 else 1
    scale_y = (max_y - min_y) / height if (max_y - min_y) > 0 else 1
    
    grid = np.zeros((height, width), dtype=int)
    marked = set()
    
    for x, y in points:
        gx = int((x - min_x) / scale_x)
        gy = int((y - min_y) / scale_y)
        
        gx_spaced = (gx // spacing) * spacing
        gy_spaced = (gy // spacing) * spacing
        
        if 0 <= gx_spaced < width and 0 <= gy_spaced < height:
            if (gx_spaced, gy_spaced) not in marked:
                grid[gy_spaced, gx_spaced] = 1
                marked.add((gx_spaced, gy_spaced))
    
    metadata = {
        'min_x': float(min_x), 'min_y': float(min_y),
        'max_x': float(max_x), 'max_y': float(max_y),
        'scale_x': scale_x, 'scale_y': scale_y,
        'width': width, 'height': height,
        'spacing': spacing
    }
    
    return grid, metadata


# Example usage (uncomment after extracting points):
# grid, metadata = points_to_grid(points, GRID_WIDTH, GRID_HEIGHT, POINT_SPACING)
# print(f"✓ Grid size: {grid.shape}")
# print(f"✓ Occupied cells: {np.sum(grid)}")
# print(f"✓ Coverage: {100*np.sum(grid)/grid.size:.1f}%\n")


# ============================================================================
# CELL 7: CENTERLINE EXTRACTION (Distance Transform - NO EXTERNAL DEPS!)
# ============================================================================
print("="*70)
print("CELL 7: Extracting Centerlines from Thick Lines")
print("="*70)

def extract_centerline(grid, min_distance=1):
    """
    Extract centerline from thick lines using distance transform
    Uses only scipy.ndimage (pre-installed on DexterOS)
    
    Args:
        grid: Binary occupancy grid (1=occupied, 0=empty)
        min_distance: Minimum distance from edge to be considered centerline
    
    Returns:
        centerline_grid: Binary grid with only centerline points
        waypoints: List of (x, y) centerline coordinates
    """
    if np.sum(grid) == 0:
        return grid, []
    
    # Distance transform: distance to nearest edge
    dist_transform = distance_transform_edt(grid)
    
    # Find ridge points (local maxima along centerline)
    centerline = (dist_transform >= min_distance).astype(int)
    
    # Extract waypoints from centerline
    waypoints = []
    ys, xs = np.where(centerline == 1)
    for x, y in zip(xs, ys):
        waypoints.append((int(x), int(y)))
    
    print(f"  Distance transform complete")
    print(f"  Found {len(waypoints)} centerline points")
    
    return centerline, waypoints


# Example usage (uncomment after creating grid):
# centerline_grid, waypoints = extract_centerline(grid, min_distance=1)
# print(f"\n✓ Centerline extraction complete")
# print(f"✓ Total waypoints: {len(waypoints)}\n")


# ============================================================================
# CELL 8: PATH PLANNING (Custom TSP - NO EXTERNAL DEPENDENCIES!)
# ============================================================================
print("="*70)
print("CELL 8: Planning Optimal Path (Custom TSP)")
print("="*70)

def calculate_distance(p1, p2):
    """Calculate Euclidean distance between two points"""
    return math.hypot(p2[0] - p1[0], p2[1] - p1[1])


def nearest_neighbor_tsp(waypoints, start):
    """
    Greedy Nearest Neighbor TSP approximation
    Pure Python implementation - no external dependencies!
    
    Args:
        waypoints: List of (x, y) tuples
        start: Starting position (x, y)
    
    Returns:
        ordered_path: List of waypoints in visit order
    """
    if len(waypoints) == 0:
        return []
    
    unvisited = set(range(len(waypoints)))
    path = []
    current = start
    
    while unvisited:
        # Find nearest unvisited waypoint
        min_dist = float('inf')
        nearest_idx = None
        
        for idx in unvisited:
            wp = waypoints[idx]
            dist = calculate_distance(current, wp)
            if dist < min_dist:
                min_dist = dist
                nearest_idx = idx
        
        # Visit nearest waypoint
        path.append(waypoints[nearest_idx])
        current = waypoints[nearest_idx]
        unvisited.remove(nearest_idx)
    
    return path


def two_opt_improve(path, max_iterations=100):
    """
    2-opt improvement for TSP path
    Pure Python implementation
    
    Args:
        path: List of waypoints
        max_iterations: Maximum optimization iterations
    
    Returns:
        improved_path: Optimized path
    """
    def path_length(p):
        total = 0
        for i in range(len(p) - 1):
            total += calculate_distance(p[i], p[i+1])
        return total
    
    improved = True
    iterations = 0
    
    while improved and iterations < max_iterations:
        improved = False
        for i in range(1, len(path) - 2):
            for j in range(i + 1, len(path)):
                if j - i == 1:
                    continue
                
                # Try swapping edges
                new_path = path[:i] + path[i:j][::-1] + path[j:]
                
                if path_length(new_path) < path_length(path):
                    path = new_path
                    improved = True
        
        iterations += 1
    
    return path


# Example usage (uncomment after extracting waypoints):
# start_position = (START_X, START_Y)
# ordered_path = nearest_neighbor_tsp(waypoints, start_position)
# ordered_path = two_opt_improve(ordered_path, max_iterations=50)
# print(f"✓ Path planning complete")
# print(f"✓ Total waypoints: {len(ordered_path)}\n")


# ============================================================================
# CELL 9: SEGMENT CLASSIFICATION (Paint vs Skip)
# ============================================================================
print("="*70)
print("CELL 9: Classifying Segments (PAINT vs SKIP)")
print("="*70)

def classify_segments(path, threshold):
    """
    Classify path segments as PAINT or SKIP
    
    Args:
        path: Ordered list of waypoints
        threshold: Distance threshold (cells)
    
    Returns:
        segments: List of (waypoint_a, waypoint_b, action) tuples
    """
    segments = []
    
    for i in range(len(path) - 1):
        wp_a = path[i]
        wp_b = path[i + 1]
        
        distance = calculate_distance(wp_a, wp_b)
        
        if distance <= threshold:
            action = "PAINT"
        else:
            action = "SKIP"
        
        segments.append((wp_a, wp_b, action))
    
    return segments


# Example usage (uncomment after path planning):
# segments = classify_segments(ordered_path, PAINT_DISTANCE_THRESHOLD)
# paint_count = sum(1 for _, _, action in segments if action == "PAINT")
# skip_count = sum(1 for _, _, action in segments if action == "SKIP")
# print(f"✓ PAINT segments: {paint_count}")
# print(f"✓ SKIP segments: {skip_count}\n")


# ============================================================================
# CELL 10: PATH VISUALIZATION
# ============================================================================
print("="*70)
print("CELL 10: Visualizing Complete Path")
print("="*70)

def visualize_path(centerline_grid, segments, start_pos):
    """Visualize the complete path plan"""
    plt.figure(figsize=(14, 14))
    
    # Show centerline grid
    plt.imshow(centerline_grid, cmap='Greys', origin='lower', alpha=0.3)
    
    # Plot path segments
    for wp_a, wp_b, action in segments:
        if action == "PAINT":
            plt.plot([wp_a[0], wp_b[0]], [wp_a[1], wp_b[1]], 
                    'g-', linewidth=2, alpha=0.7, label='PAINT' if wp_a == segments[0][0] else '')
        else:
            plt.plot([wp_a[0], wp_b[0]], [wp_a[1], wp_b[1]], 
                    'r--', linewidth=1, alpha=0.5, label='SKIP' if action == 'SKIP' and wp_a == segments[0][0] else '')
    
    # Mark start position
    plt.plot(start_pos[0], start_pos[1], 'bo', markersize=15, label='START')
    
    plt.title('Complete Path Plan\nGreen=PAINT | Red=SKIP', fontsize=16)
    plt.xlabel('Grid X (cells)', fontsize=12)
    plt.ylabel('Grid Y (cells)', fontsize=12)
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.axis('equal')
    plt.tight_layout()
    plt.show()


# Example usage:
# visualize_path(centerline_grid, segments, (START_X, START_Y))


CELL 5: Extracting Points from SVG (Built-in XML Parser)
CELL 6: Converting Points to Grid
CELL 7: Extracting Centerlines from Thick Lines
CELL 8: Planning Optimal Path (Custom TSP)
CELL 9: Classifying Segments (PAINT vs SKIP)
CELL 10: Visualizing Complete Path


In [2]:
# CELL 11: LOCALIZATION FUNCTIONS
# ============================================================================
print("="*70)
print("CELL 11: Localization System")
print("="*70)

class RobotLocalizer:
    """Robot position tracking using wheel encoders"""
    
    def __init__(self, start_x, start_y, start_heading, gpg_instance):
        self.x = start_x  # Grid coordinates
        self.y = start_y
        self.heading = start_heading  # Compass: 0=North, 90=East
        self.gpg = gpg_instance
        
        self.last_left_encoder = 0
        self.last_right_encoder = 0
        
        # Read initial encoder values
        try:
            self.last_left_encoder = self.gpg.get_motor_encoder(self.gpg.MOTOR_LEFT)
            self.last_right_encoder = self.gpg.get_motor_encoder(self.gpg.MOTOR_RIGHT)
        except:
            pass
    
    def grid_to_mm(self, grid_x, grid_y):
        """Convert grid coordinates to mm"""
        mm_x = grid_x * GRID_CELL_SIZE_CM * 10
        mm_y = grid_y * GRID_CELL_SIZE_CM * 10
        return mm_x, mm_y
    
    def mm_to_grid(self, mm_x, mm_y):
        """Convert mm to grid coordinates"""
        grid_x = mm_x / (GRID_CELL_SIZE_CM * 10)
        grid_y = mm_y / (GRID_CELL_SIZE_CM * 10)
        return grid_x, grid_y
    
    def update_position(self):
        """Update position from encoder readings"""
        try:
            # Read current encoders
            left_encoder = self.gpg.get_motor_encoder(self.gpg.MOTOR_LEFT)
            right_encoder = self.gpg.get_motor_encoder(self.gpg.MOTOR_RIGHT)
            
            # Calculate deltas
            delta_left = left_encoder - self.last_left_encoder
            delta_right = right_encoder - self.last_right_encoder
            
            # Convert to mm
            left_mm = (delta_left / 360) * math.pi * WHEEL_DIAMETER_MM
            right_mm = (delta_right / 360) * math.pi * WHEEL_DIAMETER_MM
            
            # Calculate movement
            distance_mm = (left_mm + right_mm) / 2
            heading_change = (right_mm - left_mm) / WHEEL_BASE_MM
            
            # Update heading (compass convention)
            self.heading = (self.heading + math.degrees(heading_change)) % 360
            
            # Convert heading to radians for calculation
            # 0°=North means we need to adjust: North is +Y, East is +X
            heading_rad = math.radians(90 - self.heading)  # Convert compass to math
            
            # Update position in mm
            current_x_mm, current_y_mm = self.grid_to_mm(self.x, self.y)
            current_x_mm += distance_mm * math.cos(heading_rad)
            current_y_mm += distance_mm * math.sin(heading_rad)
            
            # Convert back to grid
            self.x, self.y = self.mm_to_grid(current_x_mm, current_y_mm)
            
            # Update last encoder values
            self.last_left_encoder = left_encoder
            self.last_right_encoder = right_encoder
            
        except Exception as e:
            print(f"⚠ Position update error: {e}")
    
    def get_position(self):
        """Get current position"""
        return self.x, self.y, self.heading
    
    def reset_encoders(self):
        """Reset encoder tracking"""
        try:
            self.last_left_encoder = self.gpg.get_motor_encoder(self.gpg.MOTOR_LEFT)
            self.last_right_encoder = self.gpg.get_motor_encoder(self.gpg.MOTOR_RIGHT)
        except:
            pass


# Example usage (uncomment after hardware init):
# localizer = RobotLocalizer(START_X, START_Y, START_HEADING, gpg)
# print(f"✓ Localizer initialized\n")


# ============================================================================
# CELL 12: NAVIGATION FUNCTIONS
# ============================================================================
print("="*70)
print("CELL 12: Navigation Control System")
print("="*70)

class RobotNavigator:
    """Robot navigation and movement control"""
    
    def __init__(self, localizer, gpg_instance):
        self.localizer = localizer
        self.gpg = gpg_instance
    
    def calculate_heading_to_target(self, target_x, target_y):
        """Calculate compass heading to target"""
        current_x, current_y, _ = self.localizer.get_position()
        
        dx = target_x - current_x
        dy = target_y - current_y
        
        # Calculate angle (math convention: 0=East, 90=North)
        angle_rad = math.atan2(dy, dx)
        angle_deg = math.degrees(angle_rad)
        
        # Convert to compass (0=North, 90=East)
        compass_heading = (90 - angle_deg) % 360
        
        return compass_heading
    
    def calculate_distance_to_target(self, target_x, target_y):
        """Calculate distance to target in grid cells"""
        current_x, current_y, _ = self.localizer.get_position()
        distance = math.hypot(target_x - current_x, target_y - current_y)
        return distance
    
    def normalize_angle(self, angle):
        """Normalize angle to -180 to 180"""
        while angle > 180:
            angle -= 360
        while angle < -180:
            angle += 360
        return angle
    
    def turn_to_heading(self, target_heading):
        """Rotate robot to face target heading"""
        current_x, current_y, current_heading = self.localizer.get_position()
        
        # Calculate turn angle
        turn_angle = self.normalize_angle(target_heading - current_heading)
        
        print(f"  Turning {turn_angle:.1f}° to heading {target_heading:.1f}°")
        
        if abs(turn_angle) < HEADING_TOLERANCE_DEG:
            print("  Already facing target")
            return
        
        # Calculate encoder ticks needed
        arc_length_mm = abs(turn_angle) * (math.pi / 180) * (WHEEL_BASE_MM / 2)
        wheel_rotation_deg = (arc_length_mm / (math.pi * WHEEL_DIAMETER_MM)) * 360
        
        # Turn in place
        try:
            if turn_angle > 0:  # Turn right
                self.gpg.set_motor_dps(self.gpg.MOTOR_LEFT, ROBOT_SPEED_DPS)
                self.gpg.set_motor_dps(self.gpg.MOTOR_RIGHT, -ROBOT_SPEED_DPS)
            else:  # Turn left
                self.gpg.set_motor_dps(self.gpg.MOTOR_LEFT, -ROBOT_SPEED_DPS)
                self.gpg.set_motor_dps(self.gpg.MOTOR_RIGHT, ROBOT_SPEED_DPS)
            
            # Wait for turn to complete
            time.sleep(abs(wheel_rotation_deg) / ROBOT_SPEED_DPS)
            
            # Stop motors
            self.gpg.stop()
            
            # Update position
            self.localizer.update_position()
            
        except Exception as e:
            print(f"  ✗ Turn error: {e}")
            self.gpg.stop()
    
    def drive_straight(self, distance_cells):
        """Drive straight for specified distance"""
        distance_mm = distance_cells * GRID_CELL_SIZE_CM * 10
        
        print(f"  Driving {distance_cells:.2f} cells ({distance_mm:.1f} mm)")
        
        if distance_mm < 1:
            print("  Distance too small, skipping")
            return
        
        # Calculate time needed
        wheel_speed_mm_per_sec = (ROBOT_SPEED_DPS / 360) * math.pi * WHEEL_DIAMETER_MM
        drive_time = distance_mm / wheel_speed_mm_per_sec
        
        try:
            # Start driving
            self.gpg.set_motor_dps(self.gpg.MOTOR_LEFT, ROBOT_SPEED_DPS)
            self.gpg.set_motor_dps(self.gpg.MOTOR_RIGHT, ROBOT_SPEED_DPS)
            
            # Update position periodically
            update_interval = 1.0 / POSITION_UPDATE_HZ
            elapsed = 0
            
            while elapsed < drive_time:
                time.sleep(update_interval)
                self.localizer.update_position()
                elapsed += update_interval
            
            # Stop motors
            self.gpg.stop()
            
            # Final position update
            self.localizer.update_position()
            
        except Exception as e:
            print(f"  ✗ Drive error: {e}")
            self.gpg.stop()
    
    def move_to_waypoint(self, target_x, target_y):
        """Move robot to target waypoint"""
        current_x, current_y, current_heading = self.localizer.get_position()
        
        print(f"  Moving from ({current_x:.1f}, {current_y:.1f}) to ({target_x}, {target_y})")
        
        # Calculate target heading and distance
        target_heading = self.calculate_heading_to_target(target_x, target_y)
        distance = self.calculate_distance_to_target(target_x, target_y)
        
        # Turn to face target
        self.turn_to_heading(target_heading)
        
        # Drive to target
        self.drive_straight(distance)
        
        # Verify arrival
        final_x, final_y, _ = self.localizer.get_position()
        error_distance = math.hypot(target_x - final_x, target_y - final_y)
        error_cm = error_distance * GRID_CELL_SIZE_CM
        
        if error_cm <= POSITION_TOLERANCE_CM:
            print(f"  ✓ Reached target (error: {error_cm:.1f} cm)")
        else:
            print(f"  ⚠ Target reached with error: {error_cm:.1f} cm")


# Example usage:
# navigator = RobotNavigator(localizer, gpg)


# ============================================================================
# CELL 13: PAINTING CONTROL
# ============================================================================
print("="*70)
print("CELL 13: Painting Control System")
print("="*70)

class PaintingController:
    """Control servo for painting"""
    
    def __init__(self, servo):
        self.servo = servo
        self.is_down = False
    
    def lower_brush(self):
        """Lower brush to painting position"""
        if not self.is_down:
            try:
                self.servo.rotate_servo(SERVO_PAINT_DOWN)
                self.is_down = True
                print("  🎨 Brush DOWN (painting)")
                time.sleep(0.3)  # Allow servo to move
            except Exception as e:
                print(f"  ✗ Servo down error: {e}")
    
    def lift_brush(self):
        """Lift brush"""
        if self.is_down:
            try:
                self.servo.rotate_servo(SERVO_PAINT_UP)
                self.is_down = False
                print("  ⬆️ Brush UP (lifted)")
                time.sleep(0.3)  # Allow servo to move
            except Exception as e:
                print(f"  ✗ Servo up error: {e}")
    
    def execute_segment(self, waypoint_a, waypoint_b, action, navigator):
        """Execute movement segment with correct painting state"""
        print(f"\n--- Segment: {waypoint_a} → {waypoint_b} [{action}] ---")
        
        if action == "PAINT":
            self.lower_brush()
        else:
            self.lift_brush()
        
        # Navigate to waypoint
        navigator.move_to_waypoint(waypoint_b[0], waypoint_b[1])


# Example usage:
# painter = PaintingController(servo1)


# ============================================================================
# CELL 14: OBSTACLE DETECTION
# ============================================================================
print("="*70)
print("CELL 14: Obstacle Detection System")
print("="*70)

class ObstacleDetector:
    """Monitor distance sensor for obstacles"""
    
    def __init__(self, sensor, gpg_instance):
        self.sensor = sensor
        self.gpg = gpg_instance
        self.obstacle_detected = False
    
    def check_obstacle(self):
        """Check if obstacle is in path"""
        if self.sensor is None:
            return False
        
        try:
            distance_mm = self.sensor.read_mm()
            
            if distance_mm < OBSTACLE_STOP_DISTANCE_MM:
                return True
            return False
            
        except Exception as e:
            print(f"⚠ Sensor read error: {e}")
            return False
    
    def emergency_stop(self, painter):
        """Emergency stop - halt all movement"""
        print("\n" + "="*70)
        print("⚠️  OBSTACLE DETECTED - EMERGENCY STOP")
        print("="*70)
        
        try:
            self.gpg.stop()
            painter.lift_brush()
            self.obstacle_detected = True
        except Exception as e:
            print(f"✗ Emergency stop error: {e}")
    
    def wait_for_clearance(self):
        """Wait until obstacle is removed"""
        print("⏸️  Waiting for obstacle to clear...")
        print("   (Move obstacle or press Ctrl+C to abort)")
        
        while self.check_obstacle():
            time.sleep(OBSTACLE_CHECK_INTERVAL)
        
        print("✓ Path cleared, resuming mission\n")
        self.obstacle_detected = False


# Example usage:
# obstacle_detector = ObstacleDetector(distance_sensor, gpg)


CELL 11: Localization System
CELL 12: Navigation Control System
CELL 13: Painting Control System
CELL 14: Obstacle Detection System


In [3]:
# CELL 15: MISSION EXECUTION
# ============================================================================
print("="*70)
print("CELL 15: Mission Execution System")
print("="*70)

def run_mission(segments, navigator, painter, obstacle_detector, localizer):
    """Execute the complete painting mission"""
    
    print("\n" + "="*70)
    print("🚀 STARTING PAINTING MISSION")
    print("="*70)
    print(f"Total segments: {len(segments)}")
    print(f"Starting position: ({START_X}, {START_Y})")
    print(f"Starting heading: {START_HEADING}° (North)")
    print("="*70 + "\n")
    
    # Confirm start
    response = input("Ready to start mission? (yes/no): ").strip().lower()
    if response != 'yes':
        print("Mission aborted by user")
        return
    
    # Reset encoders
    localizer.reset_encoders()
    
    # Mission statistics
    segments_completed = 0
    paint_segments = 0
    skip_segments = 0
    start_time = time.time()
    
    try:
        for i, (waypoint_a, waypoint_b, action) in enumerate(segments):
            print(f"\n{'='*70}")
            print(f"Segment {i+1}/{len(segments)}")
            print(f"{'='*70}")
            
            # Check for obstacles
            if obstacle_detector.check_obstacle():
                obstacle_detector.emergency_stop(painter)
                obstacle_detector.wait_for_clearance()
            
            # Execute segment
            painter.execute_segment(waypoint_a, waypoint_b, action, navigator)
            
            # Update statistics
            segments_completed += 1
            if action == "PAINT":
                paint_segments += 1
            else:
                skip_segments += 1
            
            # Show progress
            progress = (i + 1) / len(segments) * 100
            print(f"Progress: {progress:.1f}% ({i+1}/{len(segments)})")
        
        # Mission complete
        elapsed_time = time.time() - start_time
        
        print("\n" + "="*70)
        print("✅ MISSION COMPLETE!")
        print("="*70)
        print(f"Segments completed: {segments_completed}/{len(segments)}")
        print(f"Paint segments: {paint_segments}")
        print(f"Skip segments: {skip_segments}")
        print(f"Total time: {elapsed_time:.1f} seconds")
        print(f"Final position: {localizer.get_position()}")
        print("="*70 + "\n")
        
        # Lift brush
        painter.lift_brush()
        
        # Save mission log
        mission_log = {
            'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
            'segments_total': len(segments),
            'segments_completed': segments_completed,
            'paint_segments': paint_segments,
            'skip_segments': skip_segments,
            'total_time_seconds': elapsed_time,
            'final_position': localizer.get_position()
        }
        
        log_file = os.path.join(OUTPUTS_DIR, 'mission_log.json')
        with open(log_file, 'w') as f:
            json.dump(mission_log, f, indent=2)
        
        print(f"✓ Mission log saved to: {log_file}\n")
        
    except KeyboardInterrupt:
        print("\n\n⚠️  Mission interrupted by user")
        navigator.gpg.stop()
        painter.lift_brush()
        
    except Exception as e:
        print(f"\n\n✗ Mission error: {e}")
        navigator.gpg.stop()
        painter.lift_brush()



CELL 15: Mission Execution System


In [4]:
# CELL 16: TESTING & DEBUG TOOLS
# ============================================================================
print("="*70)
print("CELL 16: Testing & Debug Tools")
print("="*70)

def test_servo(servo):
    """Test servo movement"""
    print("\n--- Servo Test ---")
    print("Testing servo...")
    
    try:
        print("Moving to UP position...")
        servo.rotate_servo(SERVO_PAINT_UP)
        time.sleep(1)
        
        print("Moving to DOWN position...")
        servo.rotate_servo(SERVO_PAINT_DOWN)
        time.sleep(1)
        
        print("Returning to UP position...")
        servo.rotate_servo(SERVO_PAINT_UP)
        print("✓ Servo test complete\n")
        
    except Exception as e:
        print(f"✗ Servo test failed: {e}\n")


def test_motors(gpg_instance):
    """Test individual motors"""
    print("\n--- Motor Test ---")
    print("⚠ Robot will move! Ensure clear space.")
    
    response = input("Continue? (yes/no): ").strip().lower()
    if response != 'yes':
        print("Test cancelled")
        return
    
    try:
        print("\nTesting LEFT motor...")
        gpg_instance.set_motor_dps(gpg_instance.MOTOR_LEFT, ROBOT_SPEED_DPS)
        time.sleep(1)
        gpg_instance.stop()
        time.sleep(0.5)
        
        print("Testing RIGHT motor...")
        gpg_instance.set_motor_dps(gpg_instance.MOTOR_RIGHT, ROBOT_SPEED_DPS)
        time.sleep(1)
        gpg_instance.stop()
        time.sleep(0.5)
        
        print("Testing BOTH motors...")
        gpg_instance.set_motor_dps(gpg_instance.MOTOR_LEFT, ROBOT_SPEED_DPS)
        gpg_instance.set_motor_dps(gpg_instance.MOTOR_RIGHT, ROBOT_SPEED_DPS)
        time.sleep(1)
        gpg_instance.stop()
        
        print("✓ Motor test complete\n")
        
    except Exception as e:
        print(f"✗ Motor test failed: {e}")
        gpg_instance.stop()


def test_distance_sensor(sensor):
    """Test distance sensor"""
    print("\n--- Distance Sensor Test ---")
    print("Reading distance for 5 seconds...")
    
    try:
        for i in range(10):
            distance = sensor.read_mm()
            print(f"  Reading {i+1}: {distance} mm ({distance/10:.1f} cm)")
            time.sleep(0.5)
        
        print("✓ Distance sensor test complete\n")
        
    except Exception as e:
        print(f"✗ Distance sensor test failed: {e}\n")


print("\n" + "="*70)
print("✅ ALL CODE LOADED - NO EXTERNAL DEPENDENCIES REQUIRED!")
print("="*70)
print("\n📝 Setup Instructions:")
print("1. Upload SVG files to:", DESIGNS_DIR)
print("2. In a cell, load your SVG:")
print("   svg_path = os.path.join(DESIGNS_DIR, 'your_file.svg')")
print("   points = extract_points_from_svg(svg_path)")
print("\n3. Process the design:")
print("   grid, metadata = points_to_grid(points, GRID_WIDTH, GRID_HEIGHT, POINT_SPACING)")
print("   centerline_grid, waypoints = extract_centerline(grid)")
print("   ordered_path = nearest_neighbor_tsp(waypoints, (START_X, START_Y))")
print("   ordered_path = two_opt_improve(ordered_path)")
print("   segments = classify_segments(ordered_path, PAINT_DISTANCE_THRESHOLD)")
print("\n4. Initialize robot:")
print("   localizer = RobotLocalizer(START_X, START_Y, START_HEADING, gpg)")
print("   navigator = RobotNavigator(localizer, gpg)")
print("   painter = PaintingController(servo1)")
print("   obstacle_detector = ObstacleDetector(distance_sensor, gpg)")
print("\n5. Run mission:")
print("   run_mission(segments, navigator, painter, obstacle_detector, localizer)")
print("\n✅ Ready to paint parking lots!\n")

CELL 16: Testing & Debug Tools

✅ ALL CODE LOADED - NO EXTERNAL DEPENDENCIES REQUIRED!

📝 Setup Instructions:


NameError: name 'DESIGNS_DIR' is not defined

In [7]:
svg_path = os.path.join(DESIGNS_DIR, 'road-highway-svgrepo-com.svg')  # Change filename!
points = extract_points_from_svg(svg_path, sample_distance=5)

print(f"✓ Extracted {len(points)} points from SVG")

  Parsing SVG with built-in XML parser...
  Found 769 points
✓ Extracted 769 points from SVG


In [8]:
# === CONVERT TO GRID ===
grid, metadata = points_to_grid(points, GRID_WIDTH, GRID_HEIGHT, POINT_SPACING)
print(f"✓ Grid size: {grid.shape}")
print(f"✓ Occupied cells: {np.sum(grid)}")

✓ Grid size: (100, 100)
✓ Occupied cells: 563


In [9]:
# === EXTRACT CENTERLINES ===
centerline_grid, waypoints = extract_centerline(grid, min_distance=1)

print(f"✓ Total waypoints: {len(waypoints)}")


  Distance transform complete
  Found 563 centerline points
✓ Total waypoints: 563


In [11]:
# === PATH PLANNING (TSP) ===
start_position = (START_X, START_Y)
print(f"Planning path from {start_position}...")

ordered_path = nearest_neighbor_tsp(waypoints, start_position)
ordered_path = two_opt_improve(ordered_path, max_iterations=50)

print(f"✓ Path planning complete")
print(f"✓ Total waypoints in path: {len(ordered_path)}")

Planning path from (0, 99)...


KeyboardInterrupt: 

In [ ]:
# === CLASSIFY SEGMENTS (PAINT vs SKIP) ===
segments = classify_segments(ordered_path, PAINT_DISTANCE_THRESHOLD)

paint_count = sum(1 for _, _, action in segments if action == "PAINT")
skip_count = sum(1 for _, _, action in segments if action == "SKIP")

print(f"✓ PAINT segments: {paint_count}")
print(f"✓ SKIP segments: {skip_count}")
print(f"✓ Total segments: {len(segments)}")


In [ ]:
# === VISUALIZE PATH ===
visualize_path(centerline_grid, segments, start_position)


In [41]:
# === INITIALIZE ROBOT ===
localizer = RobotLocalizer(START_X, START_Y, START_HEADING, gpg)
navigator = RobotNavigator(localizer, gpg)
painter = PaintingController(servo1)
obstacle_detector = ObstacleDetector(distance_sensor, gpg)

print("✓ All robot controllers initialized")


✓ All robot controllers initialized


In [42]:
# === RUN THE MISSION ===
run_mission(segments, navigator, painter, obstacle_detector, localizer)



🚀 STARTING PAINTING MISSION
Total segments: 42
Starting position: (0, 99)
Starting heading: 0° (North)


Segment 1/42

--- Segment: (0, 86) → (9, 86) [SKIP] ---
  Moving from (0.0, 99.0) to (9, 86)
  Turning 145.3° to heading 145.3°
  Driving 15.81 cells (316.2 mm)
  ⚠ Target reached with error: 36.1 cm
Progress: 2.4% (1/42)

Segment 2/42

--- Segment: (9, 86) → (13, 90) [SKIP] ---
  Moving from (-9.1, 85.8) to (13, 90)
  Turning -135.2° to heading 79.3°
  Driving 22.46 cells (449.2 mm)
  ⚠ Target reached with error: 64.0 cm
Progress: 4.8% (2/42)

Segment 3/42

⚠️  OBSTACLE DETECTED - EMERGENCY STOP
⏸️  Waiting for obstacle to clear...
   (Move obstacle or press Ctrl+C to abort)
✓ Path cleared, resuming mission


--- Segment: (13, 90) → (31, 72) [SKIP] ---
  Moving from (-13.2, 108.4) to (31, 72)
  Turning 139.7° to heading 129.4°
  Driving 57.23 cells (1144.5 mm)
  ⚠ Target reached with error: 148.5 cm
Progress: 7.1% (3/42)

Segment 4/42

--- Segment: (31, 72) → (36, 77) [SKIP] ---
 